## Create an LXD Container to run Docker

Adapted from https://ubuntu.com/tutorials/how-to-run-docker-inside-lxd-containers

1. Create a btrfs storage pool called "docker"

In [1]:
!lxc storage create docker btrfs

Storage pool docker created


2. Create the lxd container to run geonode

In [ ]:
!ansible-playbook create.yml

3. Create a new storage volume on the “docker” storage pool 

In [2]:
!lxc storage volume create docker dockerlib

Storage volume dockerlib created


4. Attach the dockerlib volume to the geonode container

In [3]:
!lxc config device add geonode docker disk pool=docker source=dockerlib path=/var/lib/docker

Device docker added to geonode


5. Add additional configuration so that Docker works well inside the container.

In [4]:
!lxc config set geonode security.nesting=true security.syscalls.intercept.mknod=true security.syscalls.intercept.setxattr=true

6. Restart the geonode instance

In [5]:
!lxc restart geonode

## Install Docker

In [9]:
!lxc exec geonode -- apt-get update
!lxc exec geonode -- apt-get -y install ca-certificates curl gnupg lsb-release
!lxc exec geonode -- rm -rf /usr/share/keyrings/docker-archive-keyring.gpg
!lxc exec geonode -- bash -c "curl -fsSL https://download.docker.com/linux/ubuntu/gpg | gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg"
!lxc exec geonode -- bash -c 'echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | tee /etc/apt/sources.list.d/docker.list > /dev/null'
!lxc exec geonode -- apt-get update
!lxc exec geonode -- apt-get install -y docker-ce docker-ce-cli containerd.io
!lxc exec geonode -- usermod -aG docker anernest

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://download.docker.com/linux/ubuntu jammy InRelease             
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lsb-release is already the newest version (11.1.0ubuntu4).
ca-certificates is already the newest version (20230311ubuntu0.22.04.1).
curl is already the newest version (7.81.0-1ubuntu1.15).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Hit:1 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease            
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-

## Add Stumble fileshare

In [11]:
# !lxc exec geonode -- mkdir -p /Stumble
!lxc config device add geonode stumbledir disk source=/Stumble path=/Stumble recursive=true

Device stumbledir added to geonode


In [ ]:
!lxc exec geonode -- docker compose -f /Stumble/BumbleBrain/GeoNode/docker-compose.yml run -d